In [1]:
import os
import numpy as np
import pandas as pd
import xgboost

import optuna
from xgboost import XGBClassifier, XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, root_mean_squared_error 
import joblib

logical_cores = os.cpu_count()
print(f"Number of logical CPU cores: {logical_cores}")

num_workers = max(1, logical_cores // 2)
print(f"Number of workers set to: {num_workers}")

def is_gpu_available():
    try:
        import torch
        return torch.cuda.is_available()
    except ImportError:
        return False

gpu_available = is_gpu_available()
print(f"GPU available: {gpu_available}")

print(xgboost.build_info())

Number of logical CPU cores: 16
Number of workers set to: 8
GPU available: True
{'BUILTIN_PREFETCH_PRESENT': False, 'CUDA_VERSION': [12, 4], 'DEBUG': False, 'MM_PREFETCH_PRESENT': True, 'THRUST_VERSION': [2, 3, 2], 'USE_CUDA': True, 'USE_DLOPEN_NCCL': False, 'USE_FEDERATED': False, 'USE_NCCL': False, 'USE_OPENMP': True, 'USE_RMM': False, 'libxgboost': 'C:\\Users\\ng_mi\\Anaconda\\envs\\pytorch-gpu\\Lib\\site-packages\\xgboost\\lib\\xgboost.dll'}


In [2]:
def load_or_create_ticker_df(csv_file_path):
    """
    Load the existing ticker DataFrame from a CSV file if it exists,
    otherwise create a new DataFrame with predefined column types.
    Ensure the DataFrame has the specified columns, add any missing columns,
    and rearrange the columns in alphabetical order, excluding 'Ticker_Symbol'.

    Args:
    csv_file_path (str): The path to the CSV file.

    Returns:
    pd.DataFrame: The loaded or newly created DataFrame.
    """
    # Define the column types
    column_types = {
        "Ticker_Symbol": str,
        "Best_Cov1D_Classification_Accuracy": float,
        "Best_Cov1D_Classification_Path": str,
        "Best_Cov1D_Regression_RMSE": float,
        "Best_Cov1D_Regression_Path": str,
        "Best_Hybrid_Cov1D_LSTM_Classification_Accuracy": float,
        "Best_Hybrid_Cov1D_LSTM_Classification_Path": str,
        "Best_Hybrid_Cov1D_LSTM_Regression_RMSE": float,
        "Best_Hybrid_Cov1D_LSTM_Regression_Path": str,
        "Best_Hybrid_Cov1D_Transformer_Classification_Accuracy": float,
        "Best_Hybrid_Cov1D_Transformer_Classification_Path": str,
        "Best_Hybrid_Cov1D_Transformer_Regression_RMSE": float,
        "Best_Hybrid_Cov1D_Transformer_Regression_Path": str,
        "Best_LSTM_Classification_Accuracy": float,
        "Best_LSTM_Classification_Path": str,
        "Best_LSTM_Regression_RMSE": float,
        "Best_LSTM_Regression_Path": str,
        "Best_Transformer_Classification_Accuracy": float,
        "Best_TransformerClassification_Path": str,
        "Best_Transformer_Regression_RMSE": float,
        "Best_Transformer_Regression_Path": str,
        "Best_XGBClassifier_Classification_Accuracy": float,
        "Best_XGBClassifier_Classification_Path": str,
        "Best_XGBRegressor_Regression_RMSE": float,
        "Best_XGBRegressor_Regression_Path": str
    }


    if os.path.isfile(csv_file_path):
        # Load the existing file into a DataFrame
        ticker_df = pd.read_csv(csv_file_path)
        
        # Ensure all specified columns are present
        for column, dtype in column_types.items():
            if column not in ticker_df.columns:
                ticker_df[column] = pd.Series(dtype=dtype)
        
        # Reorder columns alphabetically, excluding 'Ticker_Symbol'
        columns = ["Ticker_Symbol"] + sorted([col for col in ticker_df.columns if col != "Ticker_Symbol"])
        ticker_df = ticker_df[columns]
    else:
        # Create a new DataFrame with the specified column types
        ticker_df = pd.DataFrame(columns=column_types.keys()).astype(column_types)
    
    return ticker_df

csv_file_path = "../ticker-best-model.csv"
ticker_df = load_or_create_ticker_df(csv_file_path)

In [3]:
os.makedirs('../feature-importances/xbclassifier', exist_ok=True)
os.makedirs('../feature-importances/xbregressor', exist_ok=True)
os.makedirs('../models/xgboost/xbclassifier', exist_ok=True)
os.makedirs('../models/xgboost/xbregressor', exist_ok=True)

path = '../data/train'

ticker_list = []

if os.path.exists(path):
    ticker_list = [os.path.splitext(f)[0] for f in os.listdir(path) if f.endswith('.csv')]


In [4]:
def preprocess_data(df):
    if df.isna().sum().sum() > 0 or df.isin([float('inf'), float('-inf')]).sum().sum() > 0:
        df = df.replace([float('inf'), float('-inf')], float('nan')).dropna()

    df = df.dropna()

    columns_to_drop = [
        'NEXT_DAY_CLOSEPRICE', 'DAILY_CLOSEPRICE_CHANGE', 'CLOSEPRICE_DIRECTION',
        'DAILY_MIDPRICE', 'NEXT_DAY_MIDPRICE', 'DAILY_MIDPRICE_CHANGE', 'MIDPRICE_DIRECTION', 'Date'
    ]
    X = df.drop(columns=columns_to_drop)
    y_classifier = (df['DAILY_CLOSEPRICE_CHANGE'] > 0).astype(int)
    y_regressor = df['DAILY_CLOSEPRICE_CHANGE']

    return X, y_classifier, y_regressor

In [5]:
def process_xbclassifier(X, y, gpu_available, ticker, ticker_df, csv_file_path):
    def objective(trial):
        param = {
            'verbosity': 0,
            'objective': 'binary:logistic',
            'eval_metric': 'logloss',
            'tree_method': 'hist',
            'device': 'cuda' if gpu_available else 'cpu',  
            'use_label_encoder': False,
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'gamma': trial.suggest_float('gamma', 0, 5),
            'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),  # Adjusting range
            'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),     # Adjusting range
            'early_stopping_rounds': 50 
        }

        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
        model = XGBClassifier(**param)
        model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)
        preds = model.predict(X_valid)
        accuracy = accuracy_score(y_valid, preds)
        return accuracy

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=100)

    best_model = XGBClassifier(**study.best_params)
    best_model.fit(X, y, eval_set=[(X, y)], verbose=False)

    model_path = f'../models/xgboost/xbclassifier/{ticker}_best_model.pkl'

    # Update ticker_df
    if ticker in ticker_df['Ticker_Symbol'].values:
        current_score = ticker_df.loc[ticker_df['Ticker_Symbol'] == ticker, 'Best_XGBClassifier_Classification_Accuracy'].values[0]
        if pd.isnull(current_score) or study.best_value > current_score:
            ticker_df.loc[ticker_df['Ticker_Symbol'] == ticker, ['Best_XGBClassifier_Classification_Accuracy', 'Best_XGBClassifier_Classification_Path']] = [study.best_value, model_path]
            joblib.dump(best_model, model_path)
            ticker_df.to_csv(csv_file_path, index=False)
            print(f"Best model for {ticker} saved with accuracy: {study.best_value}")
             # Save feature importances
            feature_importances = pd.DataFrame(best_model.feature_importances_, index=X.columns, columns=['importance'])
            feature_importances = feature_importances.sort_values(by='importance', ascending=False)
            feature_importances.to_csv(f'../feature-importances/xbclassifier/{ticker}_feature_importances.csv')
            print(f"Feature importances for {ticker} saved.")
        else:
            print(f"Previous model accuracy: {current_score} is better for {ticker} than accuracy: {study.best_value}")
    else:
        new_row = pd.DataFrame({'Ticker_Symbol': [ticker], 'Best_XGBClassifier_Classification_Accuracy': [study.best_value], 'Best_XGBClassifier_Classification_Path': [model_path]})
        ticker_df = pd.concat([ticker_df, new_row], ignore_index=True)
        joblib.dump(best_model, model_path)
        ticker_df.to_csv(csv_file_path, index=False)
        print(f"Best model for {ticker} saved with accuracy: {study.best_value}")

        # Save feature importances
        feature_importances = pd.DataFrame(best_model.feature_importances_, index=X.columns, columns=['importance'])
        feature_importances = feature_importances.sort_values(by='importance', ascending=False)
        feature_importances.to_csv(f'../feature-importances/xbclassifier/{ticker}_feature_importances.csv')
        print(f"Feature importances for {ticker} saved.")

    return ticker_df


def process_xbregressor(X, y, gpu_available, ticker, ticker_df, csv_file_path):
    def objective(trial):
        param = {
            'verbosity': 0,
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'tree_method': 'hist',
            'device': 'cuda' if gpu_available else 'cpu',  
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'gamma': trial.suggest_float('gamma', 0, 5),
            'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),  # Adjusting range
            'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),     # Adjusting range
            'early_stopping_rounds': 50 
        }

        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
        model = XGBRegressor(**param)
        model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)
        preds = model.predict(X_valid)
        rmse = root_mean_squared_error (y_valid, preds)
        return rmse

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)

    best_model = XGBRegressor(**study.best_params)
    best_model.fit(X, y, eval_set=[(X, y)], verbose=False)

    model_path = f'../models/xgboost/xbclassifier/{ticker}_best_model.pkl'
    # Update ticker_df
    if ticker in ticker_df['Ticker_Symbol'].values:
        current_score = ticker_df.loc[ticker_df['Ticker_Symbol'] == ticker, 'Best_XGBRegressor_Regression_RMSE'].values[0]
        if pd.isnull(current_score) or study.best_value < current_score:
            ticker_df.loc[ticker_df['Ticker_Symbol'] == ticker, ['Best_XGBRegressor_Regression_RMSE', 'Best_XGBRegressor_Regression_Path']] = [study.best_value, model_path]
            joblib.dump(best_model, model_path)
            ticker_df.to_csv(csv_file_path, index=False)
            print(f"Best model for {ticker} saved with RMSE: {study.best_value}")

            # Save feature importances
            feature_importances = pd.DataFrame(best_model.feature_importances_, index=X.columns, columns=['importance'])
            feature_importances = feature_importances.sort_values(by='importance', ascending=False)
            feature_importances.to_csv(f'../feature-importances/xbregressor/{ticker}_feature_importances.csv')
            print(f"Feature importances for {ticker} saved.")
        else:
            print(f"Previous model MSE: {current_score} is better for {ticker} than MSE: {study.best_value}")
    else:
        new_row = pd.DataFrame({'Ticker_Symbol': [ticker], 'Best_XGBRegressor_Regression_RMSE': [study.best_value], 'Best_XGBRegressor_Regression_Path': [model_path]})
        ticker_df = pd.concat([ticker_df, new_row], ignore_index=True)
        joblib.dump(best_model, model_path)
        ticker_df.to_csv(csv_file_path, index=False)
        print(f"Best model for {ticker} saved with RMSE: {study.best_value}")
        # Save feature importances
        feature_importances = pd.DataFrame(best_model.feature_importances_, index=X.columns, columns=['importance'])
        feature_importances = feature_importances.sort_values(by='importance', ascending=False)
        feature_importances.to_csv(f'../feature-importances/xbregressor/{ticker}_feature_importances.csv')
        print(f"Feature importances for {ticker} saved.")
   
    
    return ticker_df

In [ ]:
for ticker in ticker_list:
    dataframe = pd.read_csv(f"../data/{ticker}.csv")
    X, y_classifier, y_regressor = preprocess_data(dataframe)
    ticker_df = process_xbclassifier(X, y_classifier, gpu_available, ticker, ticker_df, csv_file_path)
    ticker_df = process_xbregressor(X, y_regressor, gpu_available, ticker, ticker_df, csv_file_path)


[I 2024-09-07 13:54:53,290] A new study created in memory with name: no-name-0b7808a8-b556-46b9-8702-7a9da2513e58
[I 2024-09-07 13:54:53,824] Trial 0 finished with value: 0.5903614457831325 and parameters: {'n_estimators': 963, 'max_depth': 3, 'learning_rate': 0.12931975887232963, 'subsample': 0.9103891734480414, 'colsample_bytree': 0.6679961162914393, 'gamma': 1.9171311755756515, 'lambda': 0.00019777920117666612, 'alpha': 0.020399628088690714}. Best is trial 0 with value: 0.5903614457831325.
[I 2024-09-07 13:54:54,192] Trial 1 finished with value: 0.5301204819277109 and parameters: {'n_estimators': 934, 'max_depth': 8, 'learning_rate': 0.2393781014424757, 'subsample': 0.7335415557011442, 'colsample_bytree': 0.9763214051418929, 'gamma': 2.508976767704818, 'lambda': 6.821998207147933e-07, 'alpha': 0.022247656040943704}. Best is trial 0 with value: 0.5903614457831325.
[I 2024-09-07 13:54:54,536] Trial 2 finished with value: 0.5060240963855421 and parameters: {'n_estimators': 833, 'max_de

Best model for CL=F saved with accuracy: 0.6746987951807228
Feature importances for CL=F saved.


[I 2024-09-07 13:55:35,636] Trial 0 finished with value: 1.5613655506729884 and parameters: {'n_estimators': 499, 'max_depth': 8, 'learning_rate': 0.2344883802190484, 'subsample': 0.5932014410211552, 'colsample_bytree': 0.6971525491432334, 'gamma': 0.2422993983918631, 'lambda': 0.0001779251103431976, 'alpha': 0.00012458202000569746}. Best is trial 0 with value: 1.5613655506729884.
[I 2024-09-07 13:55:35,974] Trial 1 finished with value: 1.5570581683234312 and parameters: {'n_estimators': 214, 'max_depth': 8, 'learning_rate': 0.2531778775483602, 'subsample': 0.9230941120187808, 'colsample_bytree': 0.9024280234939079, 'gamma': 0.49423304155272907, 'lambda': 2.502890573721052e-07, 'alpha': 1.3641092700291779e-05}. Best is trial 1 with value: 1.5570581683234312.
[I 2024-09-07 13:55:36,477] Trial 2 finished with value: 1.5425981923378462 and parameters: {'n_estimators': 778, 'max_depth': 3, 'learning_rate': 0.03438409781289237, 'subsample': 0.8162389690709628, 'colsample_bytree': 0.63768183

Best model for CL=F saved with RMSE: 1.4932246812448329
Feature importances for CL=F saved.


[I 2024-09-07 13:56:27,516] Trial 0 finished with value: 0.6265060240963856 and parameters: {'n_estimators': 767, 'max_depth': 8, 'learning_rate': 0.12261001297347714, 'subsample': 0.9780493860206683, 'colsample_bytree': 0.8386657596441551, 'gamma': 4.777294550259298, 'lambda': 0.0003040134793993914, 'alpha': 3.4420438711980175e-08}. Best is trial 0 with value: 0.6265060240963856.
[I 2024-09-07 13:56:27,922] Trial 1 finished with value: 0.5180722891566265 and parameters: {'n_estimators': 204, 'max_depth': 9, 'learning_rate': 0.13277419063717422, 'subsample': 0.7244381874437684, 'colsample_bytree': 0.8340786661097188, 'gamma': 3.7594331212900527, 'lambda': 1.0239810417567506e-08, 'alpha': 0.00200139772019763}. Best is trial 0 with value: 0.6265060240963856.
[I 2024-09-07 13:56:28,269] Trial 2 finished with value: 0.5542168674698795 and parameters: {'n_estimators': 123, 'max_depth': 3, 'learning_rate': 0.09056804794348822, 'subsample': 0.5508028944133999, 'colsample_bytree': 0.5275839891

Best model for NVDA saved with accuracy: 0.6626506024096386
Feature importances for NVDA saved.


[I 2024-09-07 13:57:11,343] Trial 0 finished with value: 3.2982884612753383 and parameters: {'n_estimators': 247, 'max_depth': 9, 'learning_rate': 0.28299932915905873, 'subsample': 0.5934309472911312, 'colsample_bytree': 0.6852648194186332, 'gamma': 4.255774524448348, 'lambda': 1.1606090060585127e-08, 'alpha': 1.8265874551126264e-08}. Best is trial 0 with value: 3.2982884612753383.
[I 2024-09-07 13:57:11,637] Trial 1 finished with value: 2.969368443983079 and parameters: {'n_estimators': 865, 'max_depth': 5, 'learning_rate': 0.2802091853505337, 'subsample': 0.9810544157207575, 'colsample_bytree': 0.9184545560716515, 'gamma': 1.9144464453769299, 'lambda': 0.39118259228927804, 'alpha': 0.12899730325125175}. Best is trial 1 with value: 2.969368443983079.
[I 2024-09-07 13:57:11,970] Trial 2 finished with value: 2.9583283491366155 and parameters: {'n_estimators': 221, 'max_depth': 3, 'learning_rate': 0.060711547167193944, 'subsample': 0.608590179888834, 'colsample_bytree': 0.559243696051404